In [66]:
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### IOU

In [67]:
# def calculate_iou(true_mask, predicted_mask):
#     #convert true mask to gray scale
#     true_mask_gray=cv2.cvtColor(true_mask,cv2.COLOR_BGR2GRAY)
#     #convert predicted mask to gray scale
#     predicted_mask_gray=cv2.cvtColor(predicted_mask,cv2.COLOR_BGR2GRAY)
#     #find contours for true mask
#     true_contours,_=cv2.findContours(true_mask_gray,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
#     #find contours for predicted mask
#     predicted_contours,_=cv2.findContours(predicted_mask_gray,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
#     #initialize iou
#     iou=0
#     #calculate iou only if both true and predicted contours are present
#     if len(true_contours)>0 and len(predicted_contours)>0:
#         #get intersection contour area
#         intersection_area=cv2.contourArea(cv2.convexHull(cv2.bitwise_and(true_mask_gray,predicted_mask_gray)))

#         #get union contour area
#         union_area=cv2.contourArea(cv2.convexHull(cv2.bitwise_or(true_mask_gray,predicted_mask_gray)))

#         #calculate iou
#         iou=intersection_area/union_area

#     else:
#         iou=0

#     return iou

In [68]:
def calculate_iou(true_mask, predicted_mask):
    # Convert true mask to gray scale
    true_mask_gray = cv2.cvtColor(true_mask, cv2.COLOR_BGR2GRAY)
    # Convert predicted mask to gray scale
    predicted_mask_gray = cv2.cvtColor(predicted_mask, cv2.COLOR_BGR2GRAY)
    
    # Get intersection image
    intersection = cv2.bitwise_and(true_mask_gray, predicted_mask_gray)
    # Get union image
    union = cv2.bitwise_or(true_mask_gray, predicted_mask_gray)
    
    # Find contours in the intersection and union images
    contours_intersection, _ = cv2.findContours(intersection, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    contours_union, _ = cv2.findContours(union, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Initialize IoU
    iou = 0
    
    # Calculate IoU only if both true and predicted contours are present
    if len(contours_intersection) > 0 and len(contours_union) > 0:
        # Get intersection contour area
        intersection_area = cv2.contourArea(cv2.convexHull(np.concatenate(contours_intersection)))
        # Get union contour area
        union_area = cv2.contourArea(cv2.convexHull(np.concatenate(contours_union)))
        
        # Calculate IoU
        iou = intersection_area / union_area
    else:
        iou = 0

    return iou

### Dice

In [69]:
# #calculate dice score
# def calculate_dice_score(true_mask, predicted_mask):
#     #convert true mask to gray scale
#     true_mask_gray=cv2.cvtColor(true_mask,cv2.COLOR_BGR2GRAY)
#     #convert predicted mask to gray scale
#     predicted_mask_gray=cv2.cvtColor(predicted_mask,cv2.COLOR_BGR2GRAY)
#     #find contours for true mask
#     true_contours,_=cv2.findContours(true_mask_gray,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
#     #find contours for predicted mask
#     predicted_contours,_=cv2.findContours(predicted_mask_gray,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)
#     dice_score=0
#     #calculate dice score only if both true and predicted contours are present
#     if len(true_contours)>0 and len(predicted_contours)>0:
#         #calculate dice score
#         #intersection area
#         intersection_area=cv2.contourArea(cv2.convexHull(cv2.bitwise_and(true_mask_gray,predicted_mask_gray)))
#         #area of true mask
#         true_area=cv2.contourArea(cv2.convexHull(true_mask_gray))
#         #area of predicted mask
#         predicted_area=cv2.contourArea(cv2.convexHull(predicted_mask_gray))
#         #dice score
#         dice_score=2*intersection_area/(true_area+predicted_area)

#     else:
#         dice_score=0

#     return dice_score

In [70]:
def calculate_dice_score(true_mask, predicted_mask):
    # Convert true mask to gray scale
    true_mask_gray = cv2.cvtColor(true_mask, cv2.COLOR_BGR2GRAY)
    # Convert predicted mask to gray scale
    predicted_mask_gray = cv2.cvtColor(predicted_mask, cv2.COLOR_BGR2GRAY)
    
    # Get intersection image
    intersection = cv2.bitwise_and(true_mask_gray, predicted_mask_gray)
    
    # Find contours in the intersection and individual masks
    contours_intersection, _ = cv2.findContours(intersection, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if len(contours_intersection) == 0:
        return 0
    contours_true, _ = cv2.findContours(true_mask_gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours_pred, _ = cv2.findContours(predicted_mask_gray, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    dice_score = 0
    
    # Calculate Dice score only if both true and predicted contours are present
    if len(contours_true) > 0 and len(contours_pred) > 0:
        # Calculate intersection area
        intersection_area = cv2.contourArea(cv2.convexHull(np.concatenate(contours_intersection)))
        # Calculate area of true mask
        true_area = cv2.contourArea(cv2.convexHull(np.concatenate(contours_true)))
        # Calculate area of predicted mask
        predicted_area = cv2.contourArea(cv2.convexHull(np.concatenate(contours_pred)))
        
        # Calculate Dice score
        dice_score = 2 * intersection_area / (true_area + predicted_area)
    else:
        dice_score = 0

    return dice_score

### 

In [71]:
#pixel wise accuracy

def calculate_pixel_wise_accuracy(true_mask, predicted_mask):
    
    #convert true mask to gray scale
    true_mask_gray=cv2.cvtColor(true_mask,cv2.COLOR_BGR2GRAY)
    #convert predicted mask to gray scale
    predicted_mask_gray=cv2.cvtColor(predicted_mask,cv2.COLOR_BGR2GRAY)
    #calculate pixel wise accuracy
    #number of pixels in true mask
    true_pixels=np.sum(true_mask_gray==255)
    #number of pixels in predicted mask
    predicted_pixels=np.sum(predicted_mask_gray==255)
    #number of pixels in true mask and predicted mask
    true_predicted_pixels=np.sum((true_mask_gray==255) & (predicted_mask_gray==255))
    #pixel wise accuracy
    pixel_wise_accuracy=true_predicted_pixels/(true_pixels+predicted_pixels-true_predicted_pixels)

    return pixel_wise_accuracy

In [72]:
#
benign_predictions_dir='SAM_predictions/benign'
benign_gt_dir='Dataset_BUSI_with_GT/benign'

#file names in benign_gt_dir =benign (1)_mask.png
#file names in benign_predictions_dir =benign (1)_mask_prediction.png


list_of_benign_predictions_dir=os.listdir(benign_predictions_dir)

df=pd.DataFrame(columns=['File','IOU','Dice Score','Pixel Wise Accuracy'])

for files in list_of_benign_predictions_dir:

    #read true mask
    true_mask=cv2.imread(os.path.join(benign_gt_dir,files.replace('_mask_prediction.png','_mask.png')))
    #read predicted mask
    predicted_mask=cv2.imread(os.path.join(benign_predictions_dir,files))

    #calculate iou
    iou=calculate_iou(true_mask,predicted_mask)
    #calculate dice score
    dice_score=calculate_dice_score(true_mask,predicted_mask)
    #calculate pixel wise accuracy
    pixel_wise_accuracy=calculate_pixel_wise_accuracy(true_mask,predicted_mask)

    #dont use append method to add rows to dataframe

    df.loc[len(df)]=[files,iou,dice_score,pixel_wise_accuracy]
    


print(df)

#save the dataframe to a csv file
df.to_csv('benign_metrics.csv',index=False)

                                 File       IOU  Dice Score  \
0    benign (308)_mask_prediction.png  0.696738    0.823094   
1    benign (370)_mask_prediction.png  0.848071    0.917929   
2    benign (180)_mask_prediction.png  0.810896    0.894483   
3    benign (286)_mask_prediction.png  0.907247    0.950503   
4    benign (431)_mask_prediction.png  0.694407    0.820306   
..                                ...       ...         ...   
431  benign (328)_mask_prediction.png  0.748464    0.858532   
432  benign (229)_mask_prediction.png  0.849388    0.911084   
433  benign (121)_mask_prediction.png  0.895040    0.944308   
434  benign (227)_mask_prediction.png  0.604763    0.706204   
435   benign (93)_mask_prediction.png  0.776134    0.873959   

     Pixel Wise Accuracy  
0               0.566852  
1               0.803492  
2               0.799482  
3               0.793471  
4               0.676406  
..                   ...  
431             0.744627  
432             0.819587  


In [82]:
def calculate_metrics(benign_gt_dir, benign_predictions_dir,csv_file_name):
    list_of_benign_predictions_dir = os.listdir(benign_predictions_dir)

    df = pd.DataFrame(columns=['File', 'IOU', 'Dice Score', 'Pixel Wise Accuracy'])

    for files in list_of_benign_predictions_dir:
        # read true mask
        #print(files)
        true_mask = cv2.imread(os.path.join(benign_gt_dir, files.replace('_mask_prediction.png', '_mask.png')))
        # read predicted mask
        predicted_mask = cv2.imread(os.path.join(benign_predictions_dir, files))

        # calculate iou
        iou = calculate_iou(true_mask, predicted_mask)
        # calculate dice score
        dice_score = calculate_dice_score(true_mask, predicted_mask)
        # calculate pixel wise accuracy
        pixel_wise_accuracy = calculate_pixel_wise_accuracy(true_mask, predicted_mask)

        df.loc[len(df)] = [files, iou, dice_score, pixel_wise_accuracy]

    print(df.head())

    # save the dataframe to a csv file
    df.to_csv(csv_file_name, index=False)
    
    
    average_dice_score=np.mean(df['Dice Score'])
    average_iou=np.mean(df['IOU'])
    average_pixel_wise_accuracy=np.mean(df['Pixel Wise Accuracy'])
    
    return average_dice_score,average_iou,average_pixel_wise_accuracy

## only box prompt

In [83]:
average_dice_score,average_iou,average_pixel_wise_accuracy=calculate_metrics(benign_gt_dir,benign_predictions_dir,'benign_without_points.csv')

print('Average Dice Score:',average_dice_score)
print('Average IOU:',average_iou)
print('Average Pixel Wise Accuracy:',average_pixel_wise_accuracy)

                               File       IOU  Dice Score  Pixel Wise Accuracy
0  benign (308)_mask_prediction.png  0.696738    0.823094             0.566852
1  benign (370)_mask_prediction.png  0.848071    0.917929             0.803492
2  benign (180)_mask_prediction.png  0.810896    0.894483             0.799482
3  benign (286)_mask_prediction.png  0.907247    0.950503             0.793471
4  benign (431)_mask_prediction.png  0.694407    0.820306             0.676406
Average Dice Score: 0.8966026793007776
Average IOU: 0.8219298047116778
Average Pixel Wise Accuracy: 0.7751496027403703


In [84]:
malignant_predictions_dir='SAM_predictions/malignant'
malignant_gt_dir='Dataset_BUSI_with_GT/malignant'

average_dice_score,average_iou,average_pixel_wise_accuracy=calculate_metrics(malignant_gt_dir,malignant_predictions_dir,'malignant_without_points.csv')

print('Average Dice Score:',average_dice_score)
print('Average IOU:',average_iou)
print('Average Pixel Wise Accuracy:',average_pixel_wise_accuracy)

                                  File       IOU  Dice Score  \
0   malignant (59)_mask_prediction.png  0.745060    0.850841   
1  malignant (179)_mask_prediction.png  0.607260    0.749189   
2   malignant (80)_mask_prediction.png  0.584726    0.731595   
3   malignant (40)_mask_prediction.png  0.837069    0.910793   
4   malignant (18)_mask_prediction.png  0.813139    0.894438   

   Pixel Wise Accuracy  
0             0.702999  
1             0.554343  
2             0.456828  
3             0.817822  
4             0.665809  
Average Dice Score: 0.8480665238266606
Average IOU: 0.7496548193711751
Average Pixel Wise Accuracy: 0.7083671832802122


## with box and point prompt

In [85]:
benign_gt_dir='Dataset_BUSI_with_GT/benign'
benign_point_box_dir='SAM_predictions_box_point/benign'

average_dice_score,average_iou,average_pixel_wise_accuracy=calculate_metrics(benign_gt_dir,benign_point_box_dir,'benign_with_points.csv')

print('Average Dice Score:',average_dice_score)
print('Average IOU:',average_iou)
print('Average Pixel Wise Accuracy:',average_pixel_wise_accuracy)

                               File       IOU  Dice Score  Pixel Wise Accuracy
0  benign (308)_mask_prediction.png  0.436090    0.617439             0.589831
1  benign (370)_mask_prediction.png  0.851793    0.920149             0.806679
2  benign (180)_mask_prediction.png  0.705160    0.826318             0.832801
3  benign (286)_mask_prediction.png  0.910893    0.952451             0.801011
4  benign (431)_mask_prediction.png  0.402398    0.575636             0.592038
Average Dice Score: 0.6989419138979304
Average IOU: 0.597465348760627
Average Pixel Wise Accuracy: 0.6440335914096582


In [86]:
malignant_gt_dir='Dataset_BUSI_with_GT/malignant'
malignant_point_box_dir='SAM_predictions_box_point/malignant'

average_dice_score,average_iou,average_pixel_wise_accuracy=calculate_metrics(malignant_gt_dir,malignant_point_box_dir,'malignant_with_points.csv')

print('Average Dice Score:',average_dice_score)
print('Average IOU:',average_iou)
print('Average Pixel Wise Accuracy:',average_pixel_wise_accuracy)


                                  File       IOU  Dice Score  \
0   malignant (59)_mask_prediction.png  0.857074    0.920847   
1  malignant (179)_mask_prediction.png  0.673970    0.803866   
2   malignant (80)_mask_prediction.png  0.602213    0.745789   
3   malignant (40)_mask_prediction.png  0.900093    0.947058   
4   malignant (18)_mask_prediction.png  0.740771    0.826091   

   Pixel Wise Accuracy  
0             0.817334  
1             0.654354  
2             0.589297  
3             0.849670  
4             0.513867  
Average Dice Score: 0.7903546180927549
Average IOU: 0.6829500801098968
Average Pixel Wise Accuracy: 0.6772106009750228
